In [1]:
import sys
import pathlib
pth=str(pathlib.Path().absolute())
sys.path.append(('\\').join(pth.split('\\')[:-1])+"\\Utils")
from Utilities import *

In [2]:
def get_SD_by_format(f,ind1,ind2,j):
    x=np.mean(f[ind1,ind2,j.reshape(-1,1)],axis=1)
    u=np.std(f[ind1,ind2,j.reshape(-1,1)],axis=1)
    perc=np.percentile(f[ind1,ind2,j.reshape(-1,1)],np.array([0,25,50,75,100]),axis=1)
    hist=np.vectorize(pyfunc=(lambda t:np.histogram(f[ind1,ind2,t],bins=50,range=(0,255))),
                  signature='()->(j),(k)')(j)
    Mo=np.vectorize(pyfunc=(lambda x,y:y[np.where(x==np.max(x))]),signature='(j),(k)->()')(hist[0],hist[1])
    return np.vstack((x,u,perc,Mo))
v_get_SD_by_format=np.vectorize(pyfunc=get_SD_by_format,signature='(x,y,z),(a),(b),(j)->(q,w)')

def get_Hist_by_format(f,ind1,ind2,j):
    hist=np.vectorize(pyfunc=(lambda t:np.histogram(f[ind1,ind2,t],bins=50,range=(0,255))),
                  signature='()->(j),(k)')(j)
    return hist
v_get_Hist_by_format=np.vectorize(pyfunc=get_Hist_by_format,signature='(x,y,z),(a),(b),(j)->(q),(w)')

def pack_segments(DIRID,f,i):
    indx=np.where(f[:,:,9]==i)
    SD_rgb_hsv_lab=v_get_SD_by_format(f,indx[0],indx[1],dims)
    DIRID[i]['rgb_mean']=SD_rgb_hsv_lab[0,0]
    DIRID[i]['rgb_std']=SD_rgb_hsv_lab[0,1]
    DIRID[i]['rgb_per']=SD_rgb_hsv_lab[0,2]
    DIRID[i]['rgb_mo']=SD_rgb_hsv_lab[0,3]
    DIRID[i]['hsv_mean']=SD_rgb_hsv_lab[1,0]
    DIRID[i]['hsv_std']=SD_rgb_hsv_lab[1,1]
    DIRID[i]['hsv_per']=SD_rgb_hsv_lab[1,2]
    DIRID[i]['hsv_mo']=SD_rgb_hsv_lab[1,3]
    DIRID[i]['lab_mean']=SD_rgb_hsv_lab[2,0]
    DIRID[i]['lab_std']=SD_rgb_hsv_lab[2,1]
    DIRID[i]['lab_per']=SD_rgb_hsv_lab[2,2]
    DIRID[i]['lab_mo']=SD_rgb_hsv_lab[2,3]
    return 0
v_pack_segments=np.vectorize(pyfunc=pack_segments,signature="(),(x,y,z),()->()")

In [3]:
def get_Statistical_Descriptors(img,mask,n_segments=800):
    lum = np.mean(mask,axis=2).astype(int)
    mask1=lum>0

    m_slic = slic(img, n_segments=n_segments,sigma=5,slic_zero=True,mask=mask1)

    fig = plt.figure("Superpixels -- %d segments" % (500),figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(mark_boundaries(img, m_slic))
    plt.axis("off")
    
    RID=np.unique(m_slic.flatten())
    f=np.zeros((img.shape[0],img.shape[1],3+3+3+1))
    f[:,:,0:3]=img[:,:,0:3]
    f[:,:,3:6]=color.rgb2hsv(img)[:,:,0:3]
    f[:,:,6:9]=color.rgb2lab(img)[:,:,0:3]
    f[:,:,9]=m_slic

    DIRID={int(i):{'rbg_mean':np.zeros((3)),'rgb_std':np.zeros((3)),'rgb_per':np.zeros((3)),'rgb_mo':np.zeros((3)),
                   'lab_mean':np.zeros((3)),'lab_std':np.zeros((3)),'lab_per':np.zeros((3)),'lab_mo':np.zeros((3)),
                   'hsv_mean':np.zeros((3)),'hsv_std':np.zeros((3)),'hsv_per':np.zeros((3)),'hsv_mo':np.zeros((3)),
                  } for i in RID}

    v_pack_segments(DIRID,f,RID)
    return DIRID

In [4]:
name='c1anemia-300'
di=('\\').join(pth.split('\\')[:-2])+'\\Data_base\\Imagenes_originales\\'+name+'.jpg'
ROI=('\\').join(pth.split('\\')[:-2])+'\\Data_base\\Sem_Auto\\eye_'+name+'.jpg'
img = io.imread(di)
ROI = io.imread(ROI)

In [5]:
f1=open(('\\').join(pth.split('\\')[:-2])+"\\Data_base\\validcrop.txt","r")
lines=f1.readlines()
linesn=np.array(lines)
linesn=np.delete(lines,np.where(linesn=="\n"))
linesn=linesn.reshape(-1,3)
linesnc=v_replace_err(linesn)
linesnc=np.vectorize(pyfunc=lambda x:np.array([x[0].split('\n')[0]]),signature="(n)->(m)")(linesnc.reshape(-1,1)).reshape(-1,3)
xywh=linesnc[:,:2]
imgnames=linesnc[:,2]
xywh=v_no_spaces(xywh)
xywh=np.vectorize(pyfunc=(lambda x:float(x)))(xywh.reshape(1,-1)[0])
xywh=xywh.reshape(-1,4).astype(int)+1

mr=xywh[np.where(imgnames==(name+".jpg"))][0]
mask=assemble_mask(mr,img,ROI)

In [6]:
lum = np.mean(mask,axis=2).astype(int)
mask1=lum>0
m_slic = slic(img, n_segments=10,sigma=5,mask=mask1)

In [7]:
RID=np.unique(m_slic.flatten())
f=np.zeros((img.shape[0],img.shape[1],3+3+3+3+1))
f[:,:,0:3]=img[:,:,0:3]
f[:,:,3:6]=color.rgb2hsv(img)[:,:,0:3]
f[:,:,6:9]=color.rgb2lab(img)[:,:,0:3]
LCh=toLCh(img)
f[:,:,9:12]=LCh[:,:,0:3]
f[:,:,12]=m_slic

MemoryError: Unable to allocate 99.0 MiB for an array with shape (4160, 3120) and data type float64

In [5]:
lab=color.rgb2lab(img)

In [22]:
def toLCh(img):
    lab=color.rgb2lab(img)
    L=lab[:,:,0]
    a=lab[:,:,1]
    b=lab[:,:,2]
    C=(a**2+b**2)**0.5
    h=np.arctan2(b,a)
    Lch=np.concatenate((L.reshape(L.shape[0],L.shape[1],1),
                        C.reshape(L.shape[0],L.shape[1],1),
                        h.reshape(L.shape[0],L.shape[1],1)),axis=2)

In [3]:
LCh=toLCh(img)

In [5]:

toLCh(img)[:,:,0:3]

TypeError: 'NoneType' object is not subscriptable

In [3]:
np.arange(1,13).reshape(-1,3)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])